In [8]:
import argparse
import os.path as osp
import glob

import numpy as np
import mmcv
import torch
import cv2
from mmedit.apis import init_model, restoration_inference
from mmedit.core import tensor2img
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import os

config = '../configs/restorers/glean/glean_ffhq_16x.py'
checkpoint = '../pretrained_model/glean_ffhq_16x_20210527-61a3afad.pth'

video_path = '/home/jinsuyoo/mmediting/jinsu/obama/lq'
save_path = 'test_glean_obama'

#video_path = '/home/jinsuyoo/mmediting/jinsu/Zac_Efron/1.6/c_Qv_3Ya3Bg'
#save_path = 'test_glean_zac'

fps = 30

device = 0

lq_path = sorted(glob.glob(video_path + '/*.png'))

model = init_model(config, checkpoint, device=torch.device('cuda', device))

os.makedirs(save_path, exist_ok=True)

# save lq mp4
lqs = []
for img_path in lq_path:
    img = Image.open(img_path).convert('RGB')
    img = np.array(img)
    img = np.flip(img, axis=2)
    lqs.append(img)
lqs = np.stack(lqs, axis=0)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(osp.join(save_path, 'video_lq.mp4'), fourcc, fps, (64,64))
for i in range(0, lqs.shape[0]):
    img = lqs[i]
    video_writer.write(img)
video_writer.release()

results = []
for i, img_path in enumerate(lq_path):
    output = restoration_inference(model, img_path)
    output = tensor2img(output)
    mmcv.imwrite(output, osp.join(save_path, f'{i:04}_glean.png'))
    results.append(output)
results = np.stack(results, axis=0)
h, w= 1024, 1024 #results.shape[-2:]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(osp.join(save_path, 'video_glean.mp4'), fourcc, fps, (w, h))
for i in range(0, results.shape[0]):
    img = results[i]
    video_writer.write(img)
video_writer.release()



2022-03-24 19:27:35,195 - mmedit - INFO - Load pretrained model from http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


load checkpoint from http path: http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth
load checkpoint from http path: http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


2022-03-24 19:27:37,274 - mmedit - INFO - Load pretrained model from http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth
2022-03-24 19:27:39,030 - mmedit - INFO - load checkpoint from torchvision path: torchvision://vgg16


load checkpoint from local path: ../pretrained_model/glean_ffhq_16x_20210527-61a3afad.pth


2022-03-24 19:27:40,433 - mmgen - INFO - Switch to evaluation style mode: single
